In [ ]:
import pyscf

import itertools

import numpy as np
import scipy as sp

import math
import itertools

from copy import deepcopy

def dci_bruteforce(h1e = np.load("h1e.npy"), 
                   h2e = np.load("h2e.npy"), norb = 6, nelec = 6):
    
    nalpha = nelec // 2
    nbeta = nelec - nalpha

    spinless_determinants = list(itertools.combinations(range(norb), nalpha))
    # print(spinless_determinants)
    editable_spinless_determinants = [list(x) for x in spinless_determinants]

    # spinned_determinants_ppar = []
    # for i in range(len(spinless_determinants)):
    #     for j in range(len(spinless_determinants)):
    #         spinned_determinants_ppar.append((spinless_determinants[i], spinless_determinants[j]))

    # spinned_determinants_barpar = [list(x) for x in spinned_determinants_ppar]

    spinned_determinants_parbar = []
    for i in range(len(spinless_determinants)):
        for j in range(len(spinless_determinants)):
          spinned_determinants_parbar.append((editable_spinless_determinants[i], editable_spinless_determinants[j]))
    
    spinned_determinants_bbar = [list(x) for x in spinned_determinants_parbar]

    # spinless_eij

    def spinless_f(spinless_index, i, j):
        spinless_det = deepcopy(editable_spinless_determinants[spinless_index])
        if j not in spinless_det:
            return None
        else:
            spinless_det.remove(j)
            if i in spinless_det:
                return None
            else:
                spinless_det.append(i)
                spinless_det = sorted(spinless_det)
                spinless_det = tuple(spinless_det)
                output_spinless_index = spinless_determinants.index(spinless_det)
                return output_spinless_index

    def spinless_sigma(spinless_index, i, j):
        spinless_det = deepcopy(editable_spinless_determinants[spinless_index])
        # print("Indexed det: ", spinless_det)
        if j not in spinless_det:
            return None
        else:
            # spinless_det.remove(j)
            jindex = spinless_det.index(j)
            copy_spinless_det = deepcopy(spinless_det)
            copy_spinless_det.remove(j)
            if i in copy_spinless_det:
                return None
            else:
                spinless_det[jindex] = i
                # print("Updated det: ", spinless_det)
                sorted_spinless_det = sorted(spinless_det)
                # print("Sorted det: ", sorted_spinless_det)
                swaps = 0
                while sorted_spinless_det != spinless_det:
                    for k in range(len(spinless_det) - 1):
                        if spinless_det[k] > spinless_det[k+1]:
                            termi = spinless_det[k]
                            terminext = spinless_det[k+1]
                            spinless_det[k] = terminext
                            spinless_det[k+1] = termi
                            # print("Sort result: ", spinless_det)
                            swaps += 1
                # print(swaps)    
                return swaps
    
    def spinless_eij(spinless_index, i, j):
        f_result = spinless_f(spinless_index, i, j)
        if f_result is None:
            return None
        else:
            sigma_result = spinless_sigma(spinless_index, i, j)
            return (f_result, (-1)**sigma_result)
    
    spinless_eij_dict = {}
    def make_spinless_eij():
        for i in range(norb):
            for j in range(norb):
                for index in range(len(editable_spinless_determinants)):
                    eij = spinless_eij(index, i, j)
                    if eij is not None:
                        if index not in spinless_eij_dict:
                            spinless_eij_dict[index] = {}
                        spinless_eij_dict[index][(i, j)] = eij
    make_spinless_eij()

    def alpha_Eij(spinned_index, i, j):
        term = spinned_determinants_bbar[spinned_index]
        term_alpha = term[0]
        term_beta = term[1]
        
        # need to get alpha index in spinless determinants.
        alpha_index = editable_spinless_determinants.index(term_alpha)
        try:
            spinless_alpha_output = spinless_eij_dict[alpha_index][(i,j)]
        except:
            spinless_alpha_output = None
            return None
        
        if spinless_alpha_output is not None:
            new_term_alpha = editable_spinless_determinants[spinless_alpha_output[0]]
            new_term = [new_term_alpha, term_beta]
            new_index = spinned_determinants_bbar.index(new_term)
            return (new_index, spinless_alpha_output[1])
    
    def beta_Eij(spinned_index, i, j):
        term = spinned_determinants_bbar[spinned_index]
        term_alpha = term[0]
        term_beta = term[1]

        beta_index = editable_spinless_determinants.index(term_beta)
        try:
            spinless_beta_output = spinless_eij_dict[beta_index][(i,j)]
        except:
            spinless_beta_output = None
            return None
        
        if spinless_beta_output is not None:
            new_term_beta = editable_spinless_determinants[spinless_beta_output[0]]
            new_term = [term_alpha, new_term_beta]
            new_index = spinned_determinants_bbar.index(new_term)
            return (new_index, spinless_beta_output[1])
        

    alpha_Eij_dict = {}
    beta_Eij_dict = {}
    
    def make_alpha_Eij():
        for i in range(norb):
            for j in range(norb):
                for index in range(len(spinned_determinants_bbar)):
                    
                    Eij_alpha = alpha_Eij(index, i, j)
                    if Eij_alpha is not None:
                        if index not in alpha_Eij_dict:
                            alpha_Eij_dict[index] = {}
                        alpha_Eij_dict[index][(i,j)] = Eij_alpha

    def make_beta_Eij():
        for i in range(norb):
            for j in range(norb):
                for index in range(len(spinned_determinants_bbar)):
                    
                    Eij_beta = beta_Eij(index, i, j)
                    if Eij_beta is not None:
                        if index not in beta_Eij_dict:
                            beta_Eij_dict[index] = {}
                        beta_Eij_dict[index][(i,j)] = Eij_beta
    
    make_alpha_Eij()
    make_beta_Eij()


    dimension = len(spinned_determinants_bbar)

    print(dimension)
    hamiltonian_matrix = np.zeros((dimension,dimension))
    print(hamiltonian_matrix)


    def h_oneEl(h1e, h2e, alpha_dict = alpha_Eij_dict, beta_dict = beta_Eij_dict):
        gijjk = np.einsum('pqqr->pr', h2e)
        for init_index in range(dimension):
            for (i,j) in alpha_dict[init_index]:
                output_index, sign = alpha_dict[init_index][(i,j)]
                # print(output_index, sign)
                hamiltonian_matrix[init_index, output_index] += h1e[i,j] * sign
                hamiltonian_matrix[init_index, output_index] -= 0.5*gijjk[i,j] * sign
            for (i,j) in beta_dict[init_index]:
                output_index, sign = beta_dict[init_index][(i,j)]
                hamiltonian_matrix[init_index, output_index] += h1e[i,j] * sign
                hamiltonian_matrix[init_index, output_index] -= 0.5*gijjk[i,j] * sign
    
    h_oneEl(h1e, h2e)

    def h_twoEl_coulomb(h2e, alpha_dict = alpha_Eij_dict, beta_dict = beta_Eij_dict):
        # do alpha alpha terms first:
        for init_index in range(dimension):
            # need to do g_iljk E_il E_jk --> do E_jk first
            for (j,k) in alpha_dict[init_index]:
                intermediate_index, intermediate_sign = alpha_dict[init_index][(j,k)]
                for (i,l) in alpha_dict[intermediate_index]:
                    output_index, sign = alpha_dict[intermediate_index][(i,l)]
                    hamiltonian_matrix[output_index, init_index] += 0.5*h2e[i,l,j,k] * intermediate_sign * sign
                for (i,l) in beta_dict[intermediate_index]:
                    output_index, sign = beta_dict[intermediate_index][(i,l)]
                    hamiltonian_matrix[output_index, init_index] += 0.5* h2e[i,l,j,k] * intermediate_sign * sign

            for (j,k) in beta_dict[init_index]:
                intermediate_index, intermediate_sign = beta_dict[init_index][(j,k)]
                for (i,l) in alpha_dict[intermediate_index]:
                    output_index, sign = alpha_dict[intermediate_index][(i,l)]
                    hamiltonian_matrix[output_index, init_index] += 0.5* h2e[i,l,j,k] * intermediate_sign * sign
                for (i,l) in beta_dict[intermediate_index]:
                    output_index, sign = beta_dict[intermediate_index][(i,l)]
                    hamiltonian_matrix[output_index, init_index] += 0.5*h2e[i,l,j,k] * intermediate_sign * sign

    h_twoEl_coulomb(h2e)


    return hamiltonian_matrix
                    

hammy = dci_bruteforce()    

400
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [131]:
np.linalg.eigh(hammy)[0][0]

-7.839908014896353

In [132]:
# norb = 6
# nalpha = 3

# spinless_determinants = list(itertools.combinations(range(norb), nalpha))
# editable_spinless_determinants = [list(x) for x in spinless_determinants]
# print("Editable spinless determinants", editable_spinless_determinants)


#     # spinless_determinants = list(itertools.combinations(range(norb), nalpha))
#     # editable_spinless_determinants = [list(x) for x in spinless_determinants]

# spinned_determinants = []
# for i in range(len(editable_spinless_determinants)):
#     for j in range(len(editable_spinless_determinants)):
#         spinned_determinants.append((editable_spinless_determinants[i], editable_spinless_determinants[j]))

# # print(spinned_determinants)

# editable_spinned_determinants = [list(x) for x in spinned_determinants]


# spinned_determinants_bbar = deepcopy(editable_spinned_determinants)
# print("Spinned determinants bbar: ", spinned_determinants_bbar)

# def spinless_f(spinless_index, i, j):
#     spinless_det = deepcopy(editable_spinless_determinants[spinless_index])
#     if j not in spinless_det:
#         return None
#     else:
#         spinless_det.remove(j)
#         if i in spinless_det:
#             return None
#         else:
#             spinless_det.append(i)
#             spinless_det = sorted(spinless_det)
#             spinless_det = tuple(spinless_det)
#             output_spinless_index = spinless_determinants.index(spinless_det)
#             return output_spinless_index
        
# # def spinless_sigma(spinless_index, i, j):
# #     spinless_det = deepcopy(editable_spinless_determinants[spinless_index])
# #     if j not in spinless_det:
# #         return None
# #     else:
# #         spinless_det.remove(j)
# #         if i in spinless_det:
# #             return None
# #         else:
# #             spinless_det.append(i)
# #             sorted_spinless_det = sorted(spinless_det)
# #             swaps = 0
# #             while sorted_spinless_det != spinless_det:
# #                 for i in range(len(spinless_det) - 1):
# #                     if spinless_det[i] > spinless_det[i+1]:
# #                         termi = spinless_det[i]
# #                         terminext = spinless_det[i+1]
# #                         spinless_det[i] = terminext
# #                         spinless_det[i+1] = termi
# #                         swaps += 1
                
# #             return swaps

# def spinless_sigma(spinless_index, i, j):
#     spinless_det = deepcopy(editable_spinless_determinants[spinless_index])
#     # print("Indexed det: ", spinless_det)
#     if j not in spinless_det:
#         return None
#     else:
#         # spinless_det.remove(j)
#         jindex = spinless_det.index(j)
#         copy_spinless_det = deepcopy(spinless_det)
#         copy_spinless_det.remove(j)
#         if i in copy_spinless_det:
#             return None
#         else:
#             spinless_det[jindex] = i
#             # print("Updated det: ", spinless_det)
#             sorted_spinless_det = sorted(spinless_det)
#             # print("Sorted det: ", sorted_spinless_det)
#             swaps = 0
#             while sorted_spinless_det != spinless_det:
#                 for k in range(len(spinless_det) - 1):
#                     if spinless_det[k] > spinless_det[k+1]:
#                         termi = spinless_det[k]
#                         terminext = spinless_det[k+1]
#                         spinless_det[k] = terminext
#                         spinless_det[k+1] = termi
#                         # print("Sort result: ", spinless_det)
#                         swaps += 1
#             # print(swaps)    
#             return swaps

# def spinless_eij(spinless_index, i, j):
#     f_result = spinless_f(spinless_index, i, j)
#     if f_result is None:
#         return None
#     else:
#         sigma_result = spinless_sigma(spinless_index, i, j)
#         return (f_result, (-1)**sigma_result)

# spinless_eij_dict = {}
# def make_spinless_eij():
#     for i in range(norb):
#         for j in range(norb):
#             for index in range(len(editable_spinless_determinants)):
#                 eij = spinless_eij(index, i, j)
#                 if eij is not None:
#                     if index not in spinless_eij_dict:
#                         spinless_eij_dict[index] = {}
#                     spinless_eij_dict[index][(i, j)] = eij


# make_spinless_eij()

# # print(spinless_eij_dict[0])
# # for (i,j), (value,sign) in spinless_eij_dict[0].items():
# #     print(f"E({i},{j}): {value}")
# #     print(spinless_determinants[0], "\t-->", sign, "\t",spinless_determinants[np.abs(value)])
# #     print()


# def alpha_Eij(spinned_index, i, j):
#     term = spinned_determinants_bbar[spinned_index]
#     print(term)
#     term_alpha = term[0]
#     print("Alpha: ", term_alpha)
#     term_beta = term[1]
#     # print(term_beta)
    
#     # need to get alpha index in spinless determinants.
#     alpha_index = editable_spinless_determinants.index(term_alpha)
#     print(alpha_index)
#     try:
#         spinless_alpha_output = spinless_eij_dict[alpha_index][(i,j)]
#         print(spinless_alpha_output)
#     except:
#         spinless_alpha_output = None
#         return None
    
#     if spinless_alpha_output is not None:
#         new_term_alpha = editable_spinless_determinants[spinless_alpha_output[0]]
#         print(new_term_alpha)
#         new_term = [new_term_alpha, term_beta]
#         print(new_term)
#         new_index = spinned_determinants_bbar.index(new_term)
#         print(new_index)
#         print(spinned_determinants_bbar[new_index])
#         return (new_index, spinless_alpha_output[1])

# # print("Remove 0, Add 0")
# # print(alpha_Eij(1, 0, 0))


# def beta_Eij(spinned_index, i, j):
#     term = spinned_determinants_bbar[spinned_index]
#     term_alpha = term[0]
#     term_beta = term[1]

#     beta_index = editable_spinless_determinants.index(term_beta)
#     try:
#         spinless_beta_output = spinless_eij_dict[beta_index][(i,j)]
#     except:
#         spinless_beta_output = None
#         return None
    
#     if spinless_beta_output is not None:
#         new_term_beta = editable_spinless_determinants[spinless_beta_output[0]]
#         new_term = [term_alpha, new_term_beta]
#         new_index = spinned_determinants_bbar.index(new_term)
#         return (new_index, spinless_beta_output[1])



# alpha_Eij_dict = {}
# beta_Eij_dict = {}

# def make_alpha_Eij():
#     for i in range(norb):
#         for j in range(norb):
#             for index in range(len(spinned_determinants_bbar)):
                
#                 Eij_alpha = alpha_Eij(index, i, j)
#                 if Eij_alpha is not None:
#                     if index not in alpha_Eij_dict:
#                         alpha_Eij_dict[index] = {}
#                     alpha_Eij_dict[index][(i,j)] = Eij_alpha

# def make_beta_Eij():
#     for i in range(norb):
#         for j in range(norb):
#             for index in range(len(spinned_determinants_bbar)):
                
#                 Eij_beta = beta_Eij(index, i, j)
#                 if Eij_beta is not None:
#                     if index not in beta_Eij_dict:
#                         beta_Eij_dict[index] = {}
#                     beta_Eij_dict[index][(i,j)] = Eij_beta

# make_alpha_Eij()
# make_beta_Eij()


In [133]:
# #### testing dictionaries

# def test_alpha_dict(someindex, creation, annihilation):
#     alpha_output = alpha_Eij_dict[someindex][(creation,annihilation)]
#     init_det = spinned_determinants_bbar[someindex]
#     final_det = spinned_determinants_bbar[alpha_output[0]]
#     sign = alpha_output[1]
#     print(
#         "Initial: ", init_det,
#         "\n Final: ", final_det,
#         "\n Sign: ", sign
#     )

# def test_beta_dict(someindex, creation, annihilation):
#     beta_output = beta_Eij_dict[someindex][(creation,annihilation)]
#     init_det = spinned_determinants_bbar[someindex]
#     final_det = spinned_determinants_bbar[beta_output[0]]
#     sign = beta_output[1]
#     print(
#         "Initial: ", init_det,
#         "\n Final: ", final_det,
#         "\n Sign: ", sign
#     )

In [134]:
# print(spinned_determinants_bbar[0])
# for (tag1, tag2) in alpha_Eij_dict[0]:
#     print(tag1, tag2)

In [135]:
# test_alpha_dict(5, 3, 1)
# test_beta_dict(5, 3, 2)

In [40]:
# I realized I forgot to implement this with the Davidson algorithm - TODO. Oops!

import pyscf

import itertools

import numpy as np
import scipy as sp

import math
import itertools

import time

from copy import deepcopy

def dci_with_davidson(h1e = np.load("h1e.npy"), 
                   h2e = np.load("h2e.npy"), norb = 6, nelec = 6):

    start_time = time.time()
    
    nalpha = nelec // 2
    nbeta = nelec - nalpha

    spinless_determinants = list(itertools.combinations(range(norb), nalpha))
    # print(spinless_determinants)
    editable_spinless_determinants = [list(x) for x in spinless_determinants]


    spinned_determinants_parbar = []
    for i in range(len(spinless_determinants)):
        for j in range(len(spinless_determinants)):
          spinned_determinants_parbar.append((editable_spinless_determinants[i], editable_spinless_determinants[j]))
    
    spinned_determinants_bbar = [list(x) for x in spinned_determinants_parbar]



    def spinless_f(spinless_index, i, j):
        spinless_det = deepcopy(editable_spinless_determinants[spinless_index])
        if j not in spinless_det:
            return None
        else:
            spinless_det.remove(j)
            if i in spinless_det:
                return None
            else:
                spinless_det.append(i)
                spinless_det = sorted(spinless_det)
                spinless_det = tuple(spinless_det)
                output_spinless_index = spinless_determinants.index(spinless_det)
                return output_spinless_index

    def spinless_sigma(spinless_index, i, j):
        spinless_det = deepcopy(editable_spinless_determinants[spinless_index])

        if j not in spinless_det:
            return None
        else:

            jindex = spinless_det.index(j)
            copy_spinless_det = deepcopy(spinless_det)
            copy_spinless_det.remove(j)
            if i in copy_spinless_det:
                return None
            else:
                spinless_det[jindex] = i
  
                sorted_spinless_det = sorted(spinless_det)

                swaps = 0
                while sorted_spinless_det != spinless_det:
                    for k in range(len(spinless_det) - 1):
                        if spinless_det[k] > spinless_det[k+1]:
                            termi = spinless_det[k]
                            terminext = spinless_det[k+1]
                            spinless_det[k] = terminext
                            spinless_det[k+1] = termi
                            swaps += 1

                return swaps
    
    def spinless_eij(spinless_index, i, j):
        f_result = spinless_f(spinless_index, i, j)
        if f_result is None:
            return None
        else:
            sigma_result = spinless_sigma(spinless_index, i, j)
            return (f_result, (-1)**sigma_result)
    
    spinless_eij_dict = {}
    def make_spinless_eij():
        for i in range(norb):
            for j in range(norb):
                for index in range(len(editable_spinless_determinants)):
                    eij = spinless_eij(index, i, j)
                    if eij is not None:
                        if index not in spinless_eij_dict:
                            spinless_eij_dict[index] = {}
                        spinless_eij_dict[index][(i, j)] = eij
    make_spinless_eij()
    print("Spinless eij dictionary created!")
    print("Time taken to create spinless eij dictionary: ", time.time() - start_time)

    def alpha_Eij(spinned_index, i, j):
        term = spinned_determinants_bbar[spinned_index]
        term_alpha = term[0]
        term_beta = term[1]
        
        # need to get alpha index in spinless determinants.
        alpha_index = editable_spinless_determinants.index(term_alpha)
        try:
            spinless_alpha_output = spinless_eij_dict[alpha_index][(i,j)]
        except:
            spinless_alpha_output = None
            return None
        
        if spinless_alpha_output is not None:
            new_term_alpha = editable_spinless_determinants[spinless_alpha_output[0]]
            new_term = [new_term_alpha, term_beta]
            new_index = spinned_determinants_bbar.index(new_term)
            return (new_index, spinless_alpha_output[1])
    
    def beta_Eij(spinned_index, i, j):
        term = spinned_determinants_bbar[spinned_index]
        term_alpha = term[0]
        term_beta = term[1]

        beta_index = editable_spinless_determinants.index(term_beta)
        try:
            spinless_beta_output = spinless_eij_dict[beta_index][(i,j)]
        except:
            spinless_beta_output = None
            return None
        
        if spinless_beta_output is not None:
            new_term_beta = editable_spinless_determinants[spinless_beta_output[0]]
            new_term = [term_alpha, new_term_beta]
            new_index = spinned_determinants_bbar.index(new_term)
            return (new_index, spinless_beta_output[1])
        

    alpha_Eij_dict = {}
    beta_Eij_dict = {}
    
    def make_alpha_Eij():
        for i in range(norb):
            for j in range(norb):
                for index in range(len(spinned_determinants_bbar)):
                    
                    Eij_alpha = alpha_Eij(index, i, j)
                    if Eij_alpha is not None:
                        if index not in alpha_Eij_dict:
                            alpha_Eij_dict[index] = {}
                        alpha_Eij_dict[index][(i,j)] = Eij_alpha

    def make_beta_Eij():
        for i in range(norb):
            for j in range(norb):
                for index in range(len(spinned_determinants_bbar)):
                    
                    Eij_beta = beta_Eij(index, i, j)
                    if Eij_beta is not None:
                        if index not in beta_Eij_dict:
                            beta_Eij_dict[index] = {}
                        beta_Eij_dict[index][(i,j)] = Eij_beta
    
    make_alpha_Eij()
    print("Alpha Eij dictionary created!")
    print("Time taken to create alpha Eij dictionary: ", time.time() - start_time)


    make_beta_Eij()
    print("Beta Eij dictionary created!")
    print("Time taken to create beta Eij dictionary: ", time.time() - start_time)

    dimension = len(spinned_determinants_bbar)

    print(dimension)
    # hamiltonian_matrix = np.zeros((dimension,dimension))
    # print(hamiltonian_matrix)


    gijjk = np.einsum('pqqr->pr', h2e)
    print("Two-electron integrals contracted to two indices; time taken: ", time.time() - start_time)

    # As written, the two functions below construct the full Hamiltonian!
    # Unnecessary - we can approximate the ground state *without* doing this, by
    # using the Davidson algorithm.

    # def h_oneEl(h1e, h2e, alpha_dict = alpha_Eij_dict, beta_dict = beta_Eij_dict):
    #     gijjk = np.einsum('pqqr->pr', h2e)
    #     for init_index in range(dimension):
    #         for (i,j) in alpha_dict[init_index]:
    #             output_index, sign = alpha_dict[init_index][(i,j)]
    #             # print(output_index, sign)
    #             hamiltonian_matrix[init_index, output_index] += h1e[i,j] * sign
    #             hamiltonian_matrix[init_index, output_index] -= 0.5*gijjk[i,j] * sign
    #         for (i,j) in beta_dict[init_index]:
    #             output_index, sign = beta_dict[init_index][(i,j)]
    #             hamiltonian_matrix[init_index, output_index] += h1e[i,j] * sign
    #             hamiltonian_matrix[init_index, output_index] -= 0.5*gijjk[i,j] * sign
    
    # h_oneEl(h1e, h2e)

    statespace = []
    def H_c(cvector, h1e, h2e, alpha_dict = alpha_Eij_dict, beta_dict = beta_Eij_dict):
        output_vector = np.zeros((dimension, 1))

        one_electron_matrix = h1e - 0.5*gijjk

        for init_index in range(dimension):
            # let's do the one-electron part first.
            for (i,j) in alpha_dict[init_index]:
                output_index, sign = alpha_dict[init_index][(i,j)]
                output_vector[output_index][0] += cvector[init_index] * one_electron_matrix[(i,j)] * sign
            for (i,j) in beta_dict[init_index]:
                output_index, sign = beta_dict[init_index][(i,j)]
                output_vector[output_index][0] += cvector[init_index] * one_electron_matrix[(i,j)] * sign

            # now the two-electron part.
            for (j,k) in alpha_dict[init_index]:
                intermediate_index, intermediate_sign = alpha_dict[init_index][(j,k)]
                for (i,l) in alpha_dict[intermediate_index]:
                    output_index, sign = alpha_dict[intermediate_index][(i,l)]
                    output_vector[output_index][0] += cvector[init_index] * 0.5 * h2e[i,l,j,k] * intermediate_sign * sign
                for (i,l) in beta_dict[intermediate_index]:
                    output_index, sign = beta_dict[intermediate_index][(i,l)]
                    output_vector[output_index][0] += cvector[init_index] * 0.5 * h2e[i,l,j,k] * intermediate_sign * sign
            for (j,k) in beta_dict[init_index]:
                intermediate_index, intermediate_sign = beta_dict[init_index][(j,k)]
                for (i,l) in alpha_dict[intermediate_index]:
                    output_index, sign = alpha_dict[intermediate_index][(i,l)]
                    output_vector[output_index][0] += cvector[init_index] * 0.5 * h2e[i,l,j,k] * intermediate_sign * sign
                for (i,l) in beta_dict[intermediate_index]:
                    output_index, sign = beta_dict[intermediate_index][(i,l)]
                    output_vector[output_index][0] += cvector[init_index] * 0.5 * h2e[i,l,j,k] * intermediate_sign * sign

        return output_vector

    # testvector = np.full((dimension, 1), 1.0)
    # testvector /= np.linalg.norm(testvector)
    # out1 = H_c(testvector, h1e, h2e, alpha_dict = alpha_Eij_dict, beta_dict = beta_Eij_dict)
    # print(out1)

    # testvector2 = np.reshape(testvector, (dimension, ))
    # out2 = H_c(testvector2, h1e, h2e, alpha_dict = alpha_Eij_dict, beta_dict = beta_Eij_dict)
    # print(out2)

    # for i in range(dimension):
    #     print(out1[i] - out2[i])
    # return 0

    def davidson_algorithm(h1e, h2e, alpha_dict = alpha_Eij_dict, beta_dict = beta_Eij_dict):

        guessv1 = np.random.rand(dimension)
        guessv1 /= np.linalg.norm(guessv1)
        guessv2 = np.random.rand(dimension)
        guessv2 /= np.linalg.norm(guessv2)
        
        # gram schmidt
        guessv2 -= np.dot(guessv2.T, guessv1) * guessv1
        guessv2 /= np.linalg.norm(guessv2)

        guessv1 = np.reshape(guessv1, (dimension, 1))
        guessv2 = np.reshape(guessv2, (dimension, 1))
        Vectors = np.hstack((guessv1, guessv2))

        # print(Vectors)
        # print("Shape:",np.shape(Vectors))
        # initially prints (400, 2)

        converged = False

        diagonal_elements = np.zeros(dimension)
        for init_index in range(dimension):
            for (i,j) in alpha_dict[init_index]:
                output_index, sign = alpha_dict[init_index][(i,j)]
                if output_index == init_index:
                    diagonal_elements[init_index] += h1e[i,j] * sign
                    diagonal_elements[init_index] -= 0.5 * gijjk[i,j] * sign
            for (i,j) in beta_dict[init_index]:
                output_index, sign = beta_dict[init_index][(i,j)]
                if output_index == init_index:
                    diagonal_elements[init_index] += h1e[i,j] * sign
                    diagonal_elements[init_index] -= 0.5 * gijjk[i,j] * sign
            for (j,j) in alpha_dict[init_index]:
                intermediate_index, intermediate_sign = alpha_dict[init_index][(j,j)]
                for (i,i) in alpha_dict[intermediate_index]:
                    output_index, sign = alpha_dict[intermediate_index][(i,i)]
                    if output_index == init_index:
                        diagonal_elements[init_index] += 0.5 * h2e[i,i,j,j] * intermediate_sign * sign
                for (i,i) in beta_dict[intermediate_index]:
                    output_index, sign = beta_dict[intermediate_index][(i,i)]
                    if output_index == init_index:
                        diagonal_elements[init_index] += 0.5 * h2e[i,i,j,j] * intermediate_sign * sign
            for (j,j) in beta_dict[init_index]:
                intermediate_index, intermediate_sign = beta_dict[init_index][(j,j)]
                for (i,i) in alpha_dict[intermediate_index]:
                    output_index, sign = alpha_dict[intermediate_index][(i,i)]
                    if output_index == init_index:
                        diagonal_elements[init_index] += 0.5 * h2e[i,i,j,j] * intermediate_sign * sign
                for (i,i) in beta_dict[intermediate_index]:
                    output_index, sign = beta_dict[intermediate_index][(i,i)]
                    if output_index == init_index:
                        diagonal_elements[init_index] += 0.5 * h2e[i,i,j,j] * intermediate_sign * sign

        while not converged:

            # H_vectors = []
            H_vectors = None
            print(np.shape(H_vectors))
            size_of_space = Vectors.shape[1]
            # print("Size of space: ", size_of_space)
            for i in range(size_of_space):
                # print(np.shape(H_c(Vectors[:,i], h1e, h2e, alpha_dict, beta_dict)))
                if H_vectors is None:
                    H_vectors = H_c(Vectors[:,i], h1e, h2e, alpha_dict, beta_dict)
                else:
                    H_vectors = np.hstack((H_vectors, H_c(Vectors[:,i], h1e, h2e, alpha_dict, beta_dict)))
                # H_vectors.append(H_c(Vectors[:,i], h1e, h2e, alpha_dict, beta_dict))
                # print(H_vectors)
                # print(np.shape(H_vectors))
                # H_vectors = np.hstack((H_vectors, H_c(Vectors[:,i], h1e, h2e, alpha_dict, beta_dict)))
            H_vectors = np.array(H_vectors)
            H_proj = np.zeros((size_of_space, size_of_space))
            for i in range(size_of_space):
                for j in range(size_of_space):
                    # print(Vectors[:,i].T)
                    # print(H_vectors)
                    H_proj[i,j] = np.dot(Vectors[:,i].T, H_vectors[:,j])
            
            # diagonalize the projection
            theta, s = np.linalg.eigh(H_proj)
            
            # compute the residue vector
            r = np.dot((H_vectors - theta[0] * Vectors), s[:,0])
            q = -r.T / (diagonal_elements - theta[0] + 1E-18)
            q /= np.linalg.norm(q)
            q = np.reshape(q, (dimension, 1))
            eigenvector = np.dot(Vectors, s[:,0])
            eigenvalue = theta[0]
            Vectors = np.hstack((Vectors, q))
            # QR to orthogonalize
            Vectors = np.linalg.qr(Vectors)[0]

            norm = np.linalg.norm(r)
            print("Current eigenvalue: ", eigenvalue)
            print("Current norm: ", norm)
            print("Time taken for this iteration: ", time.time() - start_time)

            if norm < 0.00001:
                converged = True
                print("Converged!")
                print("Eigenvalue: ", eigenvalue)
                print("Eigenvector: ", eigenvector)
                print("Time taken for the entire Davidson algorithm: ", time.time() - start_time)
                return eigenvalue, eigenvector

    energy, vector = davidson_algorithm(h1e, h2e, alpha_dict = alpha_Eij_dict, beta_dict = beta_Eij_dict)

    # return energy, vector
    return energy
                    

ham = dci_with_davidson()

Spinless eij dictionary created!
Time taken to create spinless eij dictionary:  0.003377676010131836
Alpha Eij dictionary created!
Time taken to create alpha Eij dictionary:  0.02952885627746582
Beta Eij dictionary created!
Time taken to create beta Eij dictionary:  0.05301785469055176
400
Two-electron integrals contracted to two indices; time taken:  0.0531008243560791
()
(400, 1)
(400, 2)
Current eigenvalue:  -5.358083458027551
Current norm:  1.0068655343937658
Time taken for this iteration:  0.42261290550231934
()
(400, 1)
(400, 2)
(400, 3)
Current eigenvalue:  -6.328310146333944
Current norm:  0.8313527495530867
Time taken for this iteration:  0.8588032722473145
()
(400, 1)
(400, 2)
(400, 3)
(400, 4)
Current eigenvalue:  -6.88515676211625
Current norm:  0.7374942278510919
Time taken for this iteration:  1.4391157627105713
()
(400, 1)
(400, 2)
(400, 3)
(400, 4)
(400, 5)
Current eigenvalue:  -7.30366678529038
Current norm:  0.638858846853036
Time taken for this iteration:  2.18398737

In [ ]:
    # def h_twoEl_coulomb(h2e, alpha_dict = alpha_Eij_dict, beta_dict = beta_Eij_dict):
    #     # do alpha alpha terms first:
    #     for init_index in range(dimension):
    #         # need to do g_iljk E_il E_jk --> do E_jk first
    #         for (j,k) in alpha_dict[init_index]:
    #             intermediate_index, intermediate_sign = alpha_dict[init_index][(j,k)]
    #             for (i,l) in alpha_dict[intermediate_index]:
    #                 output_index, sign = alpha_dict[intermediate_index][(i,l)]
    #                 hamiltonian_matrix[output_index, init_index] += 0.5*h2e[i,l,j,k] * intermediate_sign * sign
    #             for (i,l) in beta_dict[intermediate_index]:
    #                 output_index, sign = beta_dict[intermediate_index][(i,l)]
    #                 hamiltonian_matrix[output_index, init_index] += 0.5* h2e[i,l,j,k] * intermediate_sign * sign

    #         for (j,k) in beta_dict[init_index]:
    #             intermediate_index, intermediate_sign = beta_dict[init_index][(j,k)]
    #             for (i,l) in alpha_dict[intermediate_index]:
    #                 output_index, sign = alpha_dict[intermediate_index][(i,l)]
    #                 hamiltonian_matrix[output_index, init_index] += 0.5* h2e[i,l,j,k] * intermediate_sign * sign
    #             for (i,l) in beta_dict[intermediate_index]:
    #                 output_index, sign = beta_dict[intermediate_index][(i,l)]
    #                 hamiltonian_matrix[output_index, init_index] += 0.5*h2e[i,l,j,k] * intermediate_sign * sign

    # h_twoEl_coulomb(h2e)

In [ ]:
# my implementation of the davidson algorithm is somewhat stupid (in regards to using QR 
# and recalculating projected Hamiltonian each time), We should try to be less stupid.

# here we attempt to do this.

import pyscf

import itertools

import numpy as np
import scipy as sp

import math
import itertools

import time

from copy import deepcopy

def dci_with_davidson_less_stupid(h1e = np.load("h1e.npy"), 
                   h2e = np.load("h2e.npy"), norb = 6, nelec = 6):

    start_time = time.time()
    
    nalpha = nelec // 2
    nbeta = nelec - nalpha

    spinless_determinants = list(itertools.combinations(range(norb), nalpha))
    # print(spinless_determinants)
    editable_spinless_determinants = [list(x) for x in spinless_determinants]


    spinned_determinants_parbar = []
    for i in range(len(spinless_determinants)):
        for j in range(len(spinless_determinants)):
          spinned_determinants_parbar.append((editable_spinless_determinants[i], editable_spinless_determinants[j]))
    
    spinned_determinants_bbar = [list(x) for x in spinned_determinants_parbar]



    def spinless_f(spinless_index, i, j):
        spinless_det = deepcopy(editable_spinless_determinants[spinless_index])
        if j not in spinless_det:
            return None
        else:
            spinless_det.remove(j)
            if i in spinless_det:
                return None
            else:
                spinless_det.append(i)
                spinless_det = sorted(spinless_det)
                spinless_det = tuple(spinless_det)
                output_spinless_index = spinless_determinants.index(spinless_det)
                return output_spinless_index

    def spinless_sigma(spinless_index, i, j):
        spinless_det = deepcopy(editable_spinless_determinants[spinless_index])

        if j not in spinless_det:
            return None
        else:

            jindex = spinless_det.index(j)
            copy_spinless_det = deepcopy(spinless_det)
            copy_spinless_det.remove(j)
            if i in copy_spinless_det:
                return None
            else:
                spinless_det[jindex] = i
  
                sorted_spinless_det = sorted(spinless_det)

                swaps = 0
                while sorted_spinless_det != spinless_det:
                    for k in range(len(spinless_det) - 1):
                        if spinless_det[k] > spinless_det[k+1]:
                            termi = spinless_det[k]
                            terminext = spinless_det[k+1]
                            spinless_det[k] = terminext
                            spinless_det[k+1] = termi
                            swaps += 1

                return swaps
    
    def spinless_eij(spinless_index, i, j):
        f_result = spinless_f(spinless_index, i, j)
        if f_result is None:
            return None
        else:
            sigma_result = spinless_sigma(spinless_index, i, j)
            return (f_result, (-1)**sigma_result)
    
    spinless_eij_dict = {}
    def make_spinless_eij():
        for i in range(norb):
            for j in range(norb):
                for index in range(len(editable_spinless_determinants)):
                    eij = spinless_eij(index, i, j)
                    if eij is not None:
                        if index not in spinless_eij_dict:
                            spinless_eij_dict[index] = {}
                        spinless_eij_dict[index][(i, j)] = eij
    make_spinless_eij()
    print("Spinless eij dictionary created!")
    print("Time taken to create spinless eij dictionary: ", time.time() - start_time)

    def alpha_Eij(spinned_index, i, j):
        term = spinned_determinants_bbar[spinned_index]
        term_alpha = term[0]
        term_beta = term[1]
        
        # need to get alpha index in spinless determinants.
        alpha_index = editable_spinless_determinants.index(term_alpha)
        try:
            spinless_alpha_output = spinless_eij_dict[alpha_index][(i,j)]
        except:
            spinless_alpha_output = None
            return None
        
        if spinless_alpha_output is not None:
            new_term_alpha = editable_spinless_determinants[spinless_alpha_output[0]]
            new_term = [new_term_alpha, term_beta]
            new_index = spinned_determinants_bbar.index(new_term)
            return (new_index, spinless_alpha_output[1])
    
    def beta_Eij(spinned_index, i, j):
        term = spinned_determinants_bbar[spinned_index]
        term_alpha = term[0]
        term_beta = term[1]

        beta_index = editable_spinless_determinants.index(term_beta)
        try:
            spinless_beta_output = spinless_eij_dict[beta_index][(i,j)]
        except:
            spinless_beta_output = None
            return None
        
        if spinless_beta_output is not None:
            new_term_beta = editable_spinless_determinants[spinless_beta_output[0]]
            new_term = [term_alpha, new_term_beta]
            new_index = spinned_determinants_bbar.index(new_term)
            return (new_index, spinless_beta_output[1])
        

    alpha_Eij_dict = {}
    beta_Eij_dict = {}
    
    def make_alpha_Eij():
        for i in range(norb):
            for j in range(norb):
                for index in range(len(spinned_determinants_bbar)):
                    
                    Eij_alpha = alpha_Eij(index, i, j)
                    if Eij_alpha is not None:
                        if index not in alpha_Eij_dict:
                            alpha_Eij_dict[index] = {}
                        alpha_Eij_dict[index][(i,j)] = Eij_alpha

    def make_beta_Eij():
        for i in range(norb):
            for j in range(norb):
                for index in range(len(spinned_determinants_bbar)):
                    
                    Eij_beta = beta_Eij(index, i, j)
                    if Eij_beta is not None:
                        if index not in beta_Eij_dict:
                            beta_Eij_dict[index] = {}
                        beta_Eij_dict[index][(i,j)] = Eij_beta
    
    make_alpha_Eij()
    print("Alpha Eij dictionary created!")
    print("Time taken to create alpha Eij dictionary: ", time.time() - start_time)


    make_beta_Eij()
    print("Beta Eij dictionary created!")
    print("Time taken to create beta Eij dictionary: ", time.time() - start_time)

    dimension = len(spinned_determinants_bbar)

    print(dimension)
    # hamiltonian_matrix = np.zeros((dimension,dimension))
    # print(hamiltonian_matrix)


    gijjk = np.einsum('pqqr->pr', h2e)
    print("Two-electron integrals contracted to two indices; time taken: ", time.time() - start_time)

    # As written, the two functions below construct the full Hamiltonian!
    # Unnecessary - we can approximate the ground state *without* doing this, by
    # using the Davidson algorithm.

    # def h_oneEl(h1e, h2e, alpha_dict = alpha_Eij_dict, beta_dict = beta_Eij_dict):
    #     gijjk = np.einsum('pqqr->pr', h2e)
    #     for init_index in range(dimension):
    #         for (i,j) in alpha_dict[init_index]:
    #             output_index, sign = alpha_dict[init_index][(i,j)]
    #             # print(output_index, sign)
    #             hamiltonian_matrix[init_index, output_index] += h1e[i,j] * sign
    #             hamiltonian_matrix[init_index, output_index] -= 0.5*gijjk[i,j] * sign
    #         for (i,j) in beta_dict[init_index]:
    #             output_index, sign = beta_dict[init_index][(i,j)]
    #             hamiltonian_matrix[init_index, output_index] += h1e[i,j] * sign
    #             hamiltonian_matrix[init_index, output_index] -= 0.5*gijjk[i,j] * sign
    
    # h_oneEl(h1e, h2e)

    statespace = []
    def H_c(cvector, h1e, h2e, alpha_dict = alpha_Eij_dict, beta_dict = beta_Eij_dict):
        output_vector = np.zeros((dimension, 1))

        one_electron_matrix = h1e - 0.5*gijjk

        for init_index in range(dimension):
            # let's do the one-electron part first.
            for (i,j) in alpha_dict[init_index]:
                output_index, sign = alpha_dict[init_index][(i,j)]
                output_vector[output_index][0] += cvector[init_index] * one_electron_matrix[(i,j)] * sign
            for (i,j) in beta_dict[init_index]:
                output_index, sign = beta_dict[init_index][(i,j)]
                output_vector[output_index][0] += cvector[init_index] * one_electron_matrix[(i,j)] * sign

            # now the two-electron part.
            for (j,k) in alpha_dict[init_index]:
                intermediate_index, intermediate_sign = alpha_dict[init_index][(j,k)]
                for (i,l) in alpha_dict[intermediate_index]:
                    output_index, sign = alpha_dict[intermediate_index][(i,l)]
                    output_vector[output_index][0] += cvector[init_index] * 0.5 * h2e[i,l,j,k] * intermediate_sign * sign
                for (i,l) in beta_dict[intermediate_index]:
                    output_index, sign = beta_dict[intermediate_index][(i,l)]
                    output_vector[output_index][0] += cvector[init_index] * 0.5 * h2e[i,l,j,k] * intermediate_sign * sign
            for (j,k) in beta_dict[init_index]:
                intermediate_index, intermediate_sign = beta_dict[init_index][(j,k)]
                for (i,l) in alpha_dict[intermediate_index]:
                    output_index, sign = alpha_dict[intermediate_index][(i,l)]
                    output_vector[output_index][0] += cvector[init_index] * 0.5 * h2e[i,l,j,k] * intermediate_sign * sign
                for (i,l) in beta_dict[intermediate_index]:
                    output_index, sign = beta_dict[intermediate_index][(i,l)]
                    output_vector[output_index][0] += cvector[init_index] * 0.5 * h2e[i,l,j,k] * intermediate_sign * sign

        return output_vector

    # testvector = np.full((dimension, 1), 1.0)
    # testvector /= np.linalg.norm(testvector)
    # out1 = H_c(testvector, h1e, h2e, alpha_dict = alpha_Eij_dict, beta_dict = beta_Eij_dict)
    # print(out1)

    # testvector2 = np.reshape(testvector, (dimension, ))
    # out2 = H_c(testvector2, h1e, h2e, alpha_dict = alpha_Eij_dict, beta_dict = beta_Eij_dict)
    # print(out2)

    # for i in range(dimension):
    #     print(out1[i] - out2[i])
    # return 0

    def davidson_algorithm(h1e, h2e, alpha_dict = alpha_Eij_dict, beta_dict = beta_Eij_dict):

        guessv1 = np.random.rand(dimension)
        guessv1 /= np.linalg.norm(guessv1)
        guessv2 = np.random.rand(dimension)
        guessv2 /= np.linalg.norm(guessv2)
        
        # gram schmidt
        guessv2 -= np.dot(guessv2.T, guessv1) * guessv1
        guessv2 /= np.linalg.norm(guessv2)

        guessv1 = np.reshape(guessv1, (dimension, 1))
        guessv2 = np.reshape(guessv2, (dimension, 1))
        Vectors = np.hstack((guessv1, guessv2))

        converged = False

        diagonal_elements = np.zeros(dimension)
        for init_index in range(dimension):
            for (i,j) in alpha_dict[init_index]:
                output_index, sign = alpha_dict[init_index][(i,j)]
                if output_index == init_index:
                    diagonal_elements[init_index] += h1e[i,j] * sign
                    diagonal_elements[init_index] -= 0.5 * gijjk[i,j] * sign
            for (i,j) in beta_dict[init_index]:
                output_index, sign = beta_dict[init_index][(i,j)]
                if output_index == init_index:
                    diagonal_elements[init_index] += h1e[i,j] * sign
                    diagonal_elements[init_index] -= 0.5 * gijjk[i,j] * sign
            for (j,j) in alpha_dict[init_index]:
                intermediate_index, intermediate_sign = alpha_dict[init_index][(j,j)]
                for (i,i) in alpha_dict[intermediate_index]:
                    output_index, sign = alpha_dict[intermediate_index][(i,i)]
                    if output_index == init_index:
                        diagonal_elements[init_index] += 0.5 * h2e[i,i,j,j] * intermediate_sign * sign
                for (i,i) in beta_dict[intermediate_index]:
                    output_index, sign = beta_dict[intermediate_index][(i,i)]
                    if output_index == init_index:
                        diagonal_elements[init_index] += 0.5 * h2e[i,i,j,j] * intermediate_sign * sign
            for (j,j) in beta_dict[init_index]:
                intermediate_index, intermediate_sign = beta_dict[init_index][(j,j)]
                for (i,i) in alpha_dict[intermediate_index]:
                    output_index, sign = alpha_dict[intermediate_index][(i,i)]
                    if output_index == init_index:
                        diagonal_elements[init_index] += 0.5 * h2e[i,i,j,j] * intermediate_sign * sign
                for (i,i) in beta_dict[intermediate_index]:
                    output_index, sign = beta_dict[intermediate_index][(i,i)]
                    if output_index == init_index:
                        diagonal_elements[init_index] += 0.5 * h2e[i,i,j,j] * intermediate_sign * sign


        H_vectors = None

        while not converged:
            size_of_space = Vectors.shape[1]
            if H_vectors is None:
                H_vectors = H_c(Vectors[:,0], h1e, h2e, alpha_dict, beta_dict)
                H_vectors = np.hstack((H_vectors, H_c(Vectors[:,1], h1e, h2e, alpha_dict, beta_dict)))
                H_vectors = np.array(H_vectors)

            else:
                H_vectors = np.hstack((H_vectors, H_c(Vectors[:,-1], h1e, h2e, alpha_dict, beta_dict)))

            H_proj = np.zeros((size_of_space, size_of_space))
            for i in range(size_of_space):
                for j in range(size_of_space):
                    H_proj[i,j] = np.dot(Vectors[:,i].T, H_vectors[:,j])
            
            # diagonalize the projection
            theta, s = np.linalg.eigh(H_proj)
            
            # compute the residue vector
            r = np.dot((H_vectors - theta[0] * Vectors), s[:,0])
            q = -r.T / (diagonal_elements - theta[0] + 1E-18)
            q /= np.linalg.norm(q)

            proj = None

            for i in range(np.shape(Vectors)[1]):
                this_vec = Vectors[:,i]
                this_vec = np.reshape(this_vec, (dimension,))
                if proj is None:
                    proj = np.dot(this_vec.T, q) * this_vec
                else:
                    proj += np.dot(this_vec.T, q) * this_vec      
            
            q -= proj
            q /= np.linalg.norm(q)
            
            eigenvector = np.dot(Vectors, s[:,0])
            eigenvalue = theta[0]
            q = np.reshape(q, (dimension, 1))
            Vectors = np.hstack((Vectors, q))


            norm = np.linalg.norm(r)
            print("Time taken for iteration: ", time.time() - start_time)

            if norm < 0.00001:
                converged = True
                print("Time taken for the entire Davidson algorithm: ", time.time() - start_time)
                return eigenvalue, eigenvector

    energy, vector = davidson_algorithm(h1e, h2e, alpha_dict = alpha_Eij_dict, beta_dict = beta_Eij_dict)

    # return energy, vector
    return energy

fin_en = dci_with_davidson_less_stupid()

Spinless eij dictionary created!
Time taken to create spinless eij dictionary:  0.005417346954345703
Alpha Eij dictionary created!
Time taken to create alpha Eij dictionary:  0.05571103096008301
Beta Eij dictionary created!
Time taken to create beta Eij dictionary:  0.09479475021362305
400
Two-electron integrals contracted to two indices; time taken:  0.09489274024963379
Current eigenvalue:  -5.330496404434471
Current norm:  1.0156521721911178
Time taken for this iteration:  0.7568399906158447
Current eigenvalue:  -6.352369470326561
Current norm:  0.8729888423158789
Time taken for this iteration:  1.0458025932312012
Current eigenvalue:  -6.9071457543249934
Current norm:  0.6769063031414543
Time taken for this iteration:  1.3418335914611816
Current eigenvalue:  -7.224820360527019
Current norm:  0.5610070567824373
Time taken for this iteration:  1.6306021213531494
Current eigenvalue:  -7.48807428214657
Current norm:  0.5474317508241553
Time taken for this iteration:  1.9065325260162354
C